Some of these boxes are heavy duty (for the pc) and some of these are just light enough

In [2]:
from transformers import pipeline

pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-en-tl")

print(pipe("This is a job for me."))

Device set to use cpu


[{'translation_text': "Ito'y isang trabaho para sa akin."}]


In [14]:
import tika
from tika import parser
import os

print(os.listdir("./booksEpub"))

epubList = []

for i in os.listdir("./booksEpub"):
    filePath = "./booksEpub/" + i
    epubList.append(parser.from_file(filePath))

['Ang_Bangka_ni_Tatay_Mario___Filipino-v12023.04.12T012033.epub', 'Ang_Paglalaro_nina_Mina_at_Manay___Filipino-v12023.06.16T022744.epub', 'Ano_ang_Kinatatakutan_nina__Anopol_at_ni_Tang-id___Filipino-v12023.06.20T194135.epub', 'Luna_Papelastika___Filipino-v12023.04.07T035616.epub', 'Misteryo_ng_Itlog___Filipino.epub', 'Pwedeng-pwede,_Pani!___Filipino-v12023.04.07T033002.epub', 'Sino_ang_Magiging_Titser_Ko_sa_Pasukan____Filipino-v12023.11.14T034013.epub', 'Si_Kokoy_at_ang_Umang-umang___Filipino-v12023.04.07T084156.epub', 'Si_Pogi,_Si_Pogs_At_Ang_Tora-tora___Filipino-v12023.11.14T082955.epub', 'Super_Wanda___Filipino-v12023.04.07T025901.epub']


In [40]:
import pymupdf
import os

print(os.listdir("./booksEpub"))

ebookContent = {}

for i in os.listdir("./booksEpub"):
    ebookText = ""
    titlePageText = ""
    filePath = "./booksEpub/" + i
    doc = pymupdf.open(filePath)
    for chapter in range(doc.chapter_count):
        print(f"Chapter {chapter}")
        if chapter==0:
            # get the texts and it will be the key
            currentPage = doc[(0,0)]
            titlePageText = currentPage.get_text()
        else:
            for page in range(doc.chapter_page_count(chapter)):
                print(f"page {page}")
                currentPage = doc[(chapter, page)]
                currentText = currentPage.get_text()
                print(currentText)
                # check if it's the last few pages
                if not(("Brought to you by" in currentText) or ("International License" in currentText) or ("Table of Contents" in currentText) or ("This book was developed as part of the ABC" in currentText) or ("Advancing Basic Education in the Philippines" in currentText)):
                    ebookText+=currentText
                else:
                    pass
    
    #print(ebookText)
    ebookContent[titlePageText] = ebookText




['Ang_Bangka_ni_Tatay_Mario___Filipino-v12023.04.12T012033.epub', 'Ang_Paglalaro_nina_Mina_at_Manay___Filipino-v12023.06.16T022744.epub', 'Ano_ang_Kinatatakutan_nina__Anopol_at_ni_Tang-id___Filipino-v12023.06.20T194135.epub', 'Luna_Papelastika___Filipino-v12023.04.07T035616.epub', 'Misteryo_ng_Itlog___Filipino.epub', 'Pwedeng-pwede,_Pani!___Filipino-v12023.04.07T033002.epub', 'Sino_ang_Magiging_Titser_Ko_sa_Pasukan____Filipino-v12023.11.14T034013.epub', 'Si_Kokoy_at_ang_Umang-umang___Filipino-v12023.04.07T084156.epub', 'Si_Pogi,_Si_Pogs_At_Ang_Tora-tora___Filipino-v12023.11.14T082955.epub', 'Super_Wanda___Filipino-v12023.04.07T025901.epub']
Chapter 0
MuPDF error: syntax error: css syntax error: expected number (OEBPS/style.css:6) (...icode-range: U+1000-109F, U>+<A9E0-A9FF, U+AA60-AA7F; /* ...)

Chapter 1
MuPDF error: syntax error: css syntax error: expected number (OEBPS/style.css:6) (...icode-range: U+1000-109F, U>+<A9E0-A9FF, U+AA60-AA7F; /* ...)

page 0
Malamig na hangin ang humaha

In [ ]:
import re

for key in ebookContent.keys():
    processedKey = re.sub(r'[\n]',' ',key)
    processedKey = re.sub(r'[\.]','',processedKey)
    processedKey = re.sub(r' ','_',processedKey)
    processedKey = re.sub(r'\?','',processedKey)
    f = open(f"./textData1_pymupdf/{processedKey}.txt","w",encoding='utf-8')
    print(ebookContent[key])
    f.writelines(ebookContent[key])
    f.close()
    print("DONE")